In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup,NavigableString, Tag
import time
import requests
import re
import csv
from selenium.webdriver.common.by import By
import pandas as pd
import os

driver = webdriver.Chrome()
    
driver.get("https://roo.cash/creditcard?source=google&medium=ad_CC_kw_g&content=kw_cc") #自動打開網頁（不要關）
    
time.sleep(5) #暫停
    
soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
#print(soup.prettify())  #輸出排版後的HTML內容

ranking_name = [] #排行榜名稱
ranking = [] #連結相關（英文）

results = soup.find_all('div', class_='tab carousel-cell text-NRooBlue-95 text-md font-md leading-6 rounded-full border border-NRooBlue-95 hover:bg-NRooBlue-10 mt-4 mr-3 lg:mt-3 lg:mr-3 py-1 px-5 whitespace-nowrap cursor-pointer')
for result in results:
    e_name = result.get('id')
    name = result.text
    print(name)
    print(e_name[4:])
    ranking_name.append(name)
    ranking.append(e_name[4:])

card_name_all = [] #全站卡名
card_link_all = [] #全站連結
card_img_all = [] #全站圖片
for i in range(len(ranking)):
    
    if i == 0: #即時排行網址特殊
        driver = webdriver.Chrome()
        driver.get("https://roo.cash/creditcard/recommendation?source=google&medium=ad_CC_kw_g&content=kw_cc") #自動打開網頁（不要關）
        time.sleep(5) #暫停
        soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
        result = soup.find('div', id='plans')  #找卡名
        titles = result.find_all("h3")
        card_name= []
        for title in titles:
            #print('☆'+title.text, end="\n")
            card_name.append(title.text)
            card_name_all.append(title.text)
            
        card_link =[] #存卡片連結
        cards = result.find_all("div", class_='plansShadow')
        for card in cards:
            card_link.append(card.find('a').get('href'))
            card_link_all.append(card.find('a').get('href'))
            
        #for i in range(len(card_link)):
            #print(card_link[i])  
            
        card_img = [] #卡圖片
        results = soup.find_all('div', class_='w-20 h-20 inline-block mr-3 lg:mr-5 flex-none')
        for result in results:
            #print(result.find('img').get('src'))
            card_img.append(result.find('img').get('src'))
            card_img_all.append(result.find('img').get('src'))
        
        card_dict = {
            '卡名':card_name,
            '卡連結':card_link,
            '卡圖片':card_img
        }
        #path = '/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv/排名csv/'    
        df = pd.DataFrame(card_dict)
        folder_name = '排名csv'
        path = "/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv"  
    
        folder_path = os.path.join(path, folder_name)
        if not os.path.isdir(folder_path):
            os.makedirs(folder_path, mode=0o777)
        path0 = str(path+'/'+folder_name+"/")
        df.to_csv(path0+"袋鼠"+ranking_name[i]+".csv", index=False) 
        
    else:
        driver = webdriver.Chrome()
        driver.get("https://roo.cash/creditcard/ranking/"+ranking[i]+"?source=google&medium=ad_CC_kw_g&content=kw_cc") #自動打開網頁（不要關）
        time.sleep(5) #暫停
        soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
        result = soup.find('div', id='plans')  #找卡名
        titles = result.find_all("h3")
        card_name= []
        for title in titles:
            #print('☆'+title.text, end="\n")
            card_name.append(title.text)
            card_name_all.append(title.text)
            
        card_link =[] #存卡片連結
        cards = result.find_all("div", class_='plansShadow')
        for card in cards:
            card_link.append(card.find('a').get('href'))
            card_link_all.append(card.find('a').get('href'))
        #for i in range(len(card_link)):
            #print(card_link[i])  
            
        card_img = [] #卡圖片
        results = soup.find_all('div', class_='w-20 h-20 inline-block mr-3 lg:mr-5 flex-none')
        for result in results:
            #print(result.find('img').get('src'))
            card_img.append(result.find('img').get('src'))
            card_img_all.append(result.find('img').get('src'))
            
        card_dict = {
            '卡名':card_name,
            '卡連結':card_link,
            '卡圖片':card_img
        }
        #path = '/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv/排名csv/' 
        
        df = pd.DataFrame(card_dict)
        folder_name = '排名csv'
        path = "/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv"  
    
        folder_path = os.path.join(path, folder_name)
        if not os.path.isdir(folder_path):
            os.makedirs(folder_path, mode=0o777)
        path0 = str(path+'/'+folder_name+"/")
        df.to_csv(path0+"袋鼠"+ranking_name[i]+".csv", index=False) 
     
              
driver.close()

card_apply_link =[] #申請連結
card_fee = [] #年費下標
card_fee1 = [] #年費內文
card_rate = [] #利率下標
card_rate1 = [] #利率內文
card_condition = [] #條件下標
card_condition1 = [] #條件內文

for i in range(len(card_link_all)): #卡連結
    headers = {'content-type': 'text/html; charset=UTF-8','user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36(KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}
    response = requests.get(card_link_all[i], headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    
    space＿f = ['一、', '二、', '三、'] #大標
    title_f = ['1. ','2. ','3. ','4. ','5.'] #中標
    
    result = soup.find('h2',class_='text-NRooBlue-100 text-2xl md:text-3xl text-center font-bold') #卡名
    print()
    print('☆'+result.text+'☆')
    print()
    
    space_list = [] #年費、特色
    title_list = [] #年費、特色
    space1_text_final = [] #內文
    
    spaces = soup.find_all('div', class_ = 'flex text-md md:text-lg align-top mb-3')
    for space in spaces:
        space_list.append(space)
        
    for j in range(len(space_list)):
        title = space_list[j].find('div', class_='text-NRooBlue-100 mr-4 whitespace-nowrap') #大標
        title_list.append(title.text)
        text_list = []
        print(space_f[j] + title_list[j], end='\n')
        print()
        results = space_list[j].find_all('div', class_ = 'text-NRooBlue-100 font-bold') #內文
        for result in results:
            text_list.append(result.text)
        text = '\n'.join(text_list)
        space1_text_final.append(text)
        for k in range(len(text_list)):
            print(title_f[k]+text_list[k], end= '\n')
        
    result = soup.find('div',onclick="openlink(event);").get('data-link')
    print('\n立即申請: \n'+str(result)+'\n')
    card_apply_link.append(str(result)) #申請連結寫入list
            
    
    s = Service(r"/usr/local/bin/chromedriver") #使用chromedriver、selenium
    driver = webdriver.Chrome(service=s)
    
    driver.get(card_link_all[i]) #自動打開網頁（不要關）
    
    time.sleep(3) #暫停
    
    soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
    
    search = soup.find('h2', class_ = 'text-NRooBlue-100 text-2xl md:text-3xl font-bold pb-1 md:pb-2')
    print('☆'+search.text+'☆') #大大標（優惠與權益）
    
    space_list = ['basic-offers mb-3','new-user-offers mb-3','responsibilities-and-rights'] #總共三區
    
    
    for j in range(len(space_list)):
        space = soup.find('div',class_=space_list[j]) #區
        if space == None : #如果有少，跳過
            continue
            
       
    #存入 
    
    #大標（一、二）
        title1 = space.find('h2', class_ = 'font-bold text-2lg md:text-xl text-NRooBlue-100 py-4 border-b-1 border-NRooGrey-60')
    #中標（1.、2.)
        title_list1 = [] 
        titles = space.find_all('div', class_ = 'offer-title mx-3 md:mx-5 text-NRooBlue-100 text-left flex-1')
        for title in titles:
            title_list1.append(title)
    #內文        
        text_list = [] 
        texts = space.find_all('span', class_ = 'text-NRooBlue-100 pb-5')
        for text in texts:
            text_list.append(text)
        
    
    #印
        print()
        print(space_f[j] + title1.text) #大標
    
        for l in range(len(title_list1)): #中標
            print()
            result = title_list1[l].find('h3')
            print(title_f[l] + result.text)
            title_list.append(result.text)
            result = title_list1[l].find('h4') #下標
            print('   '+result.text)
            
            print()
            text2 = []
            results = text_list[l].find_all('p') #內文
            for result in results:
                print(result.text, end='\n')
                text2.append(result.text)
            text = '\n'.join(text2)
            space1_text_final.append(text)
                
        if title1.text =='卡片須知':
            for l in range(len(title_list1)): #中標
                result1 = title_list1[l].find('h3')
                result = title_list1[l].find('h4') #下標
                if result1.text == '年費':
                    card_fee.append(result.text) #存入下標
                    results = text_list[l].find_all('p') #內文
                    card_text33 = []
                    for result in results:
                        card_text33.append(result.text)
                    card_fee1.append('\n'.join(card_text33)) #存入內文
                elif result1.text == '循環利率':
                    card_rate.append(result.text) #存入下標
                    results = text_list[l].find_all('p') #內文
                    card_text33 = []
                    for result in results:
                        card_text33.append(result.text)
                    card_rate1.append('\n'.join(card_text33)) #存入內文
                elif result1.text == '申辦條件':
                    card_condition.append(result.text) #存入下標
                    results = text_list[l].find_all('p') #內文
                    card_text33 = []
                    for result in results:
                        card_text33.append(result.text)
                    card_condition1.append('\n'.join(card_text33)) #存入內文
                else:
                    continue
                    
    card_dict1 = {
        '標題':title_list,
        '詳情':space1_text_final
    }
    #path = '/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv/卡優惠csv/'    
    df = pd.DataFrame(card_dict1)
    folder_name = '卡優惠csv'
    path = "/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv"  

    folder_path = os.path.join(path, folder_name)
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path, mode=0o777)
    path0 = str(path+'/'+folder_name+"/")
    df.to_csv(path0+"袋鼠"+card_name_all[i]+".csv", index=False)
    
    driver.close()

    
card_dict = {
            '卡名':card_name_all,
            '卡連結':card_link_all,
            '卡圖片':card_img_all
        }
card_dict['申請連結'] = card_apply_link  #申請連結
card_dict['年費'] = card_fee #年費下標
card_dict['年費詳細敘述'] = card_fee1 #年費內文
card_dict['循環利率'] = card_rate #利率下標
card_dict['循環利率詳細敘述'] = card_rate1 #利率內文
card_dict['申請條件'] = card_condition #條件下標
card_dict['申請條件詳細敘述'] = card_condition1 #條件內文

#path = '/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv/卡csv/'   
df = pd.DataFrame(card_dict)
folder_name = '卡csv'
path = "/Users/takoyaki/Downloads/專題爬蟲/袋鼠csv"  
folder_path = os.path.join(path, folder_name)
if not os.path.isdir(folder_path):
    os.makedirs(folder_path, mode=0o777)
path0 = str(path+'/'+folder_name+"/")
df.to_csv(path0+"袋鼠全站.csv", index=False)


☆滙豐 匯鑽卡☆

一、年費

1. 行動/電子帳單免年費
二、特色

1. 網購/ 影音娛樂/ 美食外送  指定通路最高 6% 回饋
2. icash Pay 10% OPEN POINT 回饋
3. 首次以全盈+Pay 綁定滙豐信用卡，不限金額不需登錄享 50 元全盈儲值金！

立即申請: 
https://ad.doubleclick.net/ddm/clk/561258130;370646436;f

☆優惠與權益☆

一、基本優惠

1. 現金回饋
   網購/ 影音娛樂/ 美食外送  指定通路最高 6% 回饋


【 指定外送/網購/行動支付通通最高 6%現金回饋 】

一般消費基本回饋享 1% 回饋，一般消費且達回饋達檻 + 帳戶滿額享 2％ 回饋
指定回饋平台基本享 3% 回饋，指定回饋平台且達回饋達檻 + 帳戶滿額享 6％ 回饋
指定行動支付/外送/網購回饋平台如下：街口支付、Uber Eats、momo 購物網
指定通路3%回饋每期上限NT$2,000 元，超過以1%回饋計，1%回饋無上限（註 *6%回饋翻倍 回饋達檻+帳戶滿額：消費累積現金積點 5,000 點並持有滙豐帳戶，帳戶資產餘額達新台幣 30 萬元）



【 回饋無門檻，3%回饋通路超過20家 】

超佛無門檻，1元起指定通路筆筆3%回饋；回饋達檻（5千點）+帳戶滿額（30萬）最高6%回饋（可將5千點換為1萬元刷卡金）3%回饋指定通路超過20家，行動支付/海外網購/影音串流面面俱到！指定通路回饋平台如下：
行動支付類：街口支付（帳單明細顯示街口電支）、PAYPAL
國內網購/外送類： momo購物網（不含富昇旅行社）、Uber Eats（帳單明細顯示優食台灣股份有限公司）
國外網購類（限海外交易）：AMAZON（不含Amazon web service）、ASOS、HERMES、SELFRIDGES、Shopbop.com、MYTHERESA、eBay、Farfetch、GMARKET、DOKODEMO
線上訂房類：agoda.com、Booking.com
影音串流類：Disney+、Netflix、YouTube、Spotify、Nintendo

2. 行動支付
   icash Pay 10% OPEN POINT 回饋活動｜首次以全盈+Pay 綁定滙豐信用卡，不限金額不需


☆第一銀行 御璽商旅卡☆

一、年費

1. 首年免年費，申辦電子帳單免年費
二、特色

1. 新戶好禮 首刷禮贈600元刷卡金／滿額禮享好禮二選一 2,000元刷卡金 或45,000點
2. 精選通路最高享３%現金回饋
3. 8月底前外幣最高享 3.7 %現金回饋無上限(含外幣月刷月有利活動 1.2%)

立即申請: 
https://ccard.firstbank.com.tw/cmsweb/conn/icard?cardno=ROO_VBTC

☆優惠與權益☆

一、基本優惠

1. 現金回饋
   精選通路最高享 3% 現金回饋


【 精選通路最高享 3% 現金回饋 】

活動期間：2022/7/1~2023/12/31
活動內容：於精選通路刷卡不限金額最高享3%現金回饋
精選通路：航空購票、免稅店、Klook、KKDAY、台灣高鐵、台灣大車隊(55688 APP)、大都會車隊、Uber APP、LINE TAXI、yoxi、汽車保養、購車訂金
備註：
			每戶每期帳單回饋上限500元



2. 海外消費
   外幣最高享 3.7% 現金回饋


【
御璽商旅卡外幣最高享 3.7% 現金回饋
】

外幣享2.5%現金回饋無上限
活動期間：2023.1.1~2023.8.31
活動期間內外幣月刷月有利活動最高贈6,000元(最高1.2%)
外幣月刷月有利
活動期間： 2023.1.1~2023.8.31
登錄外幣月刷月有利活動，活動期間刷外幣加碼回饋級距如下：
每期累積消費(約當臺幣金額)達3萬元(含)以上，贈回饋刷卡金300元(新臺幣)
每期累積消費(約當臺幣金額)達5萬元(含)以上，贈回饋刷卡金500元(新臺幣)
每期累積消費(約當臺幣金額)達10萬元(含)以上，贈回饋刷卡金1,000元(新臺幣)
每期累積消費(約當臺幣金額)達30萬元(含)以上，贈回饋刷卡金3,00元(新臺幣)
每期累積消費(約當臺幣金額)達50萬元(含)以上，贈回饋刷卡金6,000元(新臺幣)
注意事項：月刷月有利活動每戶每期擇優回饋乙次，每期帳單回饋上限6,000元

二、新卡友優惠

1. 袋鼠通路限定
   袋鼠金融通路首刷禮贈統一超商電子禮券 200 元

【
 袋鼠金融通路限定首刷加碼活動 】
活動期間：即日起 至 2023/12/31，並於 2024/1/31 前完


☆滙豐 現金回饋御璽卡☆

一、年費

1. 行動/電子帳單免年費
二、特色

1. 現金回饋御璽卡符合資格最高享海外 5% 現金回饋
2. icash Pay 10% OPEN POINT 回饋活動
3. 首次以全盈+Pay 綁定滙豐信用卡，不限金額不需登錄享 50 元全盈儲值金！

立即申請: 
https://ad.doubleclick.net/ddm/clk/561258130;370646436;f

☆優惠與權益☆

一、基本優惠

1. 現金回饋
   現金回饋御璽卡符合資格最高享海外5% 國內1.22%！


【 新卡友海外消費享額外1.78% 本行忠實客戶海外消費享額外2.78%現金回饋 】

活動期間： 2023.07.01~2023.09.30
回饋期間： 2023.07.01~2023.10.31
活動1. 新卡友
活動對象：滙豐現金回饋御璽卡新卡友
活動期間內線上申辦現金回饋御璽卡之新卡友並勾選使用行動或電子帳單，於核卡後至 2023/10/31期間內刷卡消費，除原回饋外，海外消費享額外1.78%現金回饋，最高享海外4%！
活動2.本行忠實客戶
活動對象：本行忠實客戶
持有本行銀行帳戶且帳戶平均帳面餘額達新台幣30萬元之忠實客戶，活動期間內線上申辦現金回饋御璽卡並勾選使用行動或電子帳單，於核卡後至 2023/10/31期間內刷卡消費，除原回饋外，海外消費享額外2.78%現金回饋，最高享海外5%！  (含原卡片權益回饋) ！

*海外含網購消費，不含歐盟及英國實體通路。
	*須勾選使用行動或電子帳單。
	* 一般新增消費(含保費)每月回饋上限1,000元。
	*現金回饋御璽卡新卡友定義: 須於核卡前六個月內未持有滙豐信用卡正卡或附卡，原持有滙豐信用卡而辦理升等、加辦或轉卡者恕不得適用。
	*詳細活動注意事項，請詳 活動官網。

2. 行動支付
   icash Pay 10% OPEN POINT 回饋活動｜首次以全盈+Pay 綁定滙豐信用卡，不限金額不需登錄享 50 元全盈儲值金！


【 不需登錄，綁定任一滙豐卡於指定網購通路享10%回饋 】

活動時間：2023.07.01~2023.12.31
活動內容：
以icash Pay綁定滙豐銀行信用卡於指定通路博客來/康是美網購/東森購物網/福容大飯店享福購/陪心寵糧網購/3TET電票通


☆富邦 momo 卡☆

一、年費

1. 首年免年費。次年免年費條件: 前一年累積消費次數達 12 次或金額達 NT$6 萬
二、特色

1. momo通路 4% 回饋無上限 
2. 千大品牌 最高 8% 回饋
3. 指定支付最高5%

立即申請: 
https://ebank.taipeifubon.com.tw/EXT/common/CAC/PLCCIndex.faces?fromSC_code=MD01

☆優惠與權益☆

一、基本優惠

1. 網購
   momo通路 4% 回饋無上限｜千大品牌最高 8% 回饋無上限


【 千大品牌 最高 8% 回饋 】

活動期間：112/4/1-112/12/31
活動方式：momo館內眾多千大品牌，刷momo卡最高8%回饋無上限
			*8%=原權益3%+限時加碼1%+品牌加碼1%~4%
			*免登錄，看更多品牌優惠及加碼時間依momo購物網公告為準


【 momo 通路 4% 回饋無上限 】

活動期間：112/4/1至112/12/31止 
活動方式：活動期間內於momo通路刷momo卡，筆筆享4%回饋無上限！
			*4%=原權益3%+限時加碼1%
			*momo通路含購物網、看看買、型錄
			*活動回饋為momo幣(1momo幣=1元)

2. 指定回饋
   店外綁定支付/錢包最高 5% 回饋

【
	店外綁定支付/錢包最高 5% 回饋】
活動期間：112/7/1-112/12/31
活動內容：國內店外通路用momo卡綁定指定支付/錢包之一般消費享最高5% momo幣回饋！(含原權益1%回饋)
*活動加碼4%回饋，加碼上限200元momo幣/戶，限量5,000名，需登錄
*國內店外通路：非momo通路(momo購物網、看看買、型錄)之新增消費
*指定支付：Apple Pay、Google Pay、Samsung Pay、LINE Pay、街口支付、Skm pay、beanfun!橘子支付、PX Pay


【五大連鎖咖啡最高10%回饋】

活動期間：112/7/1-112/12/31
活動內容：於指定五大連鎖咖啡(限獨立門市)使用momo卡綁定指定支付/錢包消費享最高10% momo幣回饋(含原權益1%回饋)
*活動加碼9%回饋，每戶加碼上限50momo幣/月，限量1,000名/月，需每月各別登


☆第一銀行 Living Green 綠活卡☆

一、年費

1. 首年免年費，申辦電子帳單免年費
二、特色

1. 新戶數位申辦最高贈 750 元
2. 健康樂活通路最高 10% 回饋
3. 自動加值/臺鐵/高鐵 5% 回饋

立即申請: 
https://ccard.firstbank.com.tw/cmsweb/conn/icard?cardno=ROO_green

☆優惠與權益☆

一、基本優惠

1. 現金回饋
   健康樂活通路最高 10% 回饋


【 健康樂活通路最高 10% 回饋 】

活動期間：2023/7/1 - 2023/12/31
活動內容：
綠活卡於「節能共享類」通路刷卡不限金額享10%刷卡金：TESLA充電費、GoShare、Gogoro Network、Gogoro、iRent、Go Smart等。
綠活卡於「有機綠活/健身運動/健康保健類」通路，單筆刷卡滿500元贈50元、單筆刷卡滿1,000元贈100元、單筆刷卡滿1,500元贈150元…，以此類推：義美食品、棉花田、馬可先生麵包坊、無毒農、World Gym、健身工廠、迪卡儂、摩曼頓運動用品、大樹藥局、佑全保健藥妝、躍獅藥局、丁丁藥局、啄木鳥藥局等。
* 需登錄，上述四類通路之回饋合併計算，每戶每月帳單回饋上限250元

2. 交通通勤
   自動加值/臺鐵/高鐵 5% 回饋


【 自動加值/臺鐵/高鐵 5% 回饋 】

活動期間： 2023/1/1 - 2023/12/31
活動內容：綠活卡當月帳單新增一般消費單筆滿1,000元(含)以上，當月帳單內之【自動加值交易】以及【臺鐵、高鐵刷卡交易】享5%現金回饋。
*自動加值交易每戶每月帳單回饋上限100元；臺鐵、高鐵刷卡交易每戶每月帳單回饋上限200元。

3. 分期零利率
   滿額享 6 期分期 0 利率


【 滿額享 6 期分期 0 利率 】

活動期間： 2023/1/1 - 2023/12/31
活動內容：事先完成登錄並刷綠活卡單筆一般消費滿5,000元(含)以上，享6期分期0利率 (請於帳單結帳日前3個工作天完成活動登錄，活動登錄完成後，其歸戶內之綠活卡單筆一般消費滿5,000元(含)以上交易將自動分6期)。

4. 指定回饋
   計算碳足跡抽宏佳騰智慧電車、刷卡金888元等好禮


【 計算碳足跡抽宏佳騰智

...


☆華南 酷愛黑卡（晶緻卡)☆

一、年費

1. 正卡年費 NT$1,200，附卡免年費
二、特色

1. 一般消費最高 2% 現金回饋
2. 加油消費 5%-15% 現金回饋
3. eTag 網路信用卡儲值享 10% 現金回饋

立即申請: 


☆優惠與權益☆

一、基本優惠

1. 現金回饋
   一般消費最高 2% 現金回饋

【一般消費 最高 2% 現金回饋】

2. 停車加油
   加油消費 5%-15% 現金回饋

【加油消費 5%-15%現金回饋】

3. 現金回饋
   eTag 網路信用卡儲值享 10% 現金回饋

【eTag網路信用卡儲值享 10% 現金回饋】

二、新卡友優惠

1. 首刷禮
   好禮六選一

【好禮六選一】

三、卡片須知

1. 年費
   正卡年費 NT$1,200，附卡免年費

正卡： NT$1,200 。附卡：免收費
首年免年費，自核卡日起，每年不限金額消費 3 筆，或申請電子帳單後每年不限金額消費 1 筆，即免收該卡次年年費。
詳情請參閱華南銀行官網信用卡年費規範。

2. 循環利率
   5.36%~15%

循環利率：5.36%~15% (基準日2018/5/1)；預借現金手續費:每筆預借現金金額X3.5%+NT$100。其他相關費率依本行網站公告。
謹慎理財 信用至上

3. 申辦條件
   年滿 18 歲有正當職業即可申請

正卡持卡人得為其配偶或年滿 15 歲以上之子女或兄弟姐妹、父母或配偶父母申請核發附卡。
您在本行申請的所有信用卡（含正、附卡）共用一個信用額度。

☆台新 新光三越聯名卡☆

一、年費

1. 普卡正卡 NT$ 300，附卡 NT$ 300
二、特色

1. 週末逛新光三越 筆筆享 1% 回饋
2. 免費市區停車每日最高 2 小時，全館滿仟免費停車 2 小時
3. 新光三越店內最新活動｜點數現場折抵消費，每 1 點台新Point折抵刷卡消費金額NT$ 1

立即申請: 
https://www.taishinbank.com.tw/eServiceA/CreditCardAP/apply/index.jsp?pc=16

☆優惠與權益☆

一、基本優惠

1. 百貨購物
   週末逛新光三越 筆筆享 1% 回饋


【週末逛新光三越 筆筆享 1% 回饋 】

活動期間：2023/1


☆中國信託 LEXUS 聯名卡☆

一、年費

1. 首年免年費，次年年費 NT$10,000
二、特色

1. 市區停車免費天天最高享 4 小時
2. LEXUS 板噴工資及 LEXUS 車主精品 9 折
3. 紅利點數最高 5 倍回饋

立即申請: 


☆優惠與權益☆

一、基本優惠

1. 停車加油
   市區停車免費天天最高享 4 小時

【市區停車免費天天最高享 4 小時】

2. 銀行紅利點數
   紅利點數最高 5 倍回饋

【紅利點數最高 5 倍回饋】

3. 指定回饋
   LEXUS 板噴工資及 LEXUS 車主精品 9 折

【LEXUS 板噴工資及 LEXUS 車主精品 9 折】

二、新卡友優惠

1. 新戶優惠
   新卡申辦 LEXUS 聯名卡享 500 元刷卡金

【新卡申辦 LEXUS 聯名卡享 500 元刷卡金】

三、卡片須知

1. 年費
   首年免年費，次年年費 NT$10,000

LEXUS 聯名卡首年免年費，次年年費 NT$10,000元，附卡免年費。
當年刷卡消費累計滿 NT$50 萬元或當年刷卡消費累計滿 NT$36 萬元，且須包含當年經銷商據點消費 2 筆，次年免收年費 （含繳稅，正、附卡消費金額合併計算)。
相關年費減免辦法及詳細收費方式請參考中國信託官方網站（信用卡利率及費用資訊)說明為準。

2. 循環利率
   5.97%~13.47%

信用卡循環年利率：中信 ARMs 指數+加碼利率（5.98%~13.47%)上限為 15%，預借現金手續費為每筆預借金額X3.5%+150 元，循環利率基準日為 104 年 9 月 1 日，其他費用請上中信官網查詢。
謹慎理財 信用至上

3. 申辦條件
   正卡申請人須為 LEXUS 車主並年滿 18 歲

LEXUS 聯名卡正卡限申請對象如下，且於申辦時須檢附 LEXUS 汽車行車執照影本及 LEXUS 車主身分證影本。
申請對象：
申請 LEXUS 聯名卡正卡者，需年滿 18 歲，固定年收入 NT$120 萬元以上，或近 2 個月平均資產餘額達 NT$150 萬元以上。
年滿 15 歲者可申請附卡，不限定 LEXUS 車主。附卡可申請者限正卡人的配偶、子女、父母或兄弟姐妹，附卡申請人如未滿 18 歲，須法定代理人（即父母/監護人)共同簽名同意。

☆台新 


☆將來 將將卡☆

一、年費

1. VISA 簽帳金融卡免年費
二、特色

1. 跨行轉帳、提領免手續費
2. 刷卡最高享 5% 回饋

立即申請: 


☆優惠與權益☆

一、基本優惠

1. 生活繳費
   跨行轉帳及提領免手續費

【跨行轉帳及提領免手續費】

2. 銀行紅利點數
   刷卡最高享 5% 回饋

【刷卡最高享 5% 回饋】

二、新卡友優惠

1. 新戶優惠
   完成指定任務拿最高 700 元回饋

【完成指定任務拿最高 NT$700 回饋】

三、卡片須知

1. 年費
   VISA 簽帳金融卡免年費

正卡： VISA 金融卡免年費
詳情請參閱將來銀行官網規範

2. 循環利率
   簽帳金融卡無循環利率

謹慎理財 信用至上

3. 申辦條件
   法定成年並領有中華民國國民身分證之本國自然人

須為法定成年人並領有中華民國國民身分證之本國自然人，憑身分證和第二身分證件（駕照、健保卡和中華民國護照）及他行臨櫃開立的一般帳戶，即可使用將來銀行 APP 進行開戶。

☆中國信託 寰遊美國運通卡☆

一、年費

1. 首年免年費、正附卡 NT$600 
二、特色

1. 保險刷卡享現金回饋1.68%
2. 量販超市與餐廳享現金回饋1%
3. 海外消費現金回饋

立即申請: 


☆優惠與權益☆

一、基本優惠

1. 保費回饋
   保險刷卡享現金回饋 1.68%

【保險刷卡享現金回饋 1.68%】

2. 生活繳費
   量販超市、餐飲享現金回饋 1%

【量販超市、餐飲享現金回饋 1%】

3. 海外消費
   海外消費現金回饋

【海外消費現金回饋】

二、新卡友優惠

1. 新戶優惠
   新卡刷滿888，送200刷卡金

【新卡刷滿888，送200刷卡金】
 

三、卡片須知

1. 年費
   首年免年費、正附卡 NT$600 

首年免年費
正附卡均 NT$600
首年免年費，於年費計算期間累積消費金額達 NT$6,000（含）以上，續享次年免年費禮遇
其餘相關年費年新增消費、減免辦法及詳細收費方式請參考本行網站說明為準

2. 循環利率
   本行 ARMs 指數＋加碼利率（5.97%~13.47%），上限為 15%

信用卡循環年利率：本行 ARMs 指數+加碼利率（5.98%~13.47%）上限為 15%，預借現金


☆滙豐 旅人無限卡☆

一、年費

1. 信用卡年費：NT$ 8,000 元
二、特色

1. 刷卡有哩最優享 10元 = 1 哩，回饋無上限
2. 刷旅人卡支付全額機票費用，享旅行不便險理賠金額最高新台幣31,000元
3. 權益哩遇｜年享免費機場接送4次與貴賓室8次

立即申請: 


☆優惠與權益☆

一、基本優惠

1. 里程回饋
   刷卡有哩最優享 10 元 = 1 哩，回饋無上限


【 刷卡有哩 每刷 NT$ 10元，向美好旅程更近 1 哩 回饋無上限】

兌換日期：2023/07/01~2023/12/31
活動內容：
於2023/01/01至2023/06/30 以信用卡消費累積之紅利積分或旅遊積分兌換／換購紅利商品及服務或轉換為飛行哩程

2. 保費回饋
   旅行無憂，首選滙豐！旅人卡旅行不便險理賠金額最高新台幣31,000元。


【 刷旅人卡支付全額機票費用，可享有專屬旅行不便保險權益！】

活動期間：即日起~2023/09/30
活動內容：若因染疫或其他不可抗力之行程取消、縮減、延期或錯過轉機，經保險公司認定，可享保障理賠金額最高新台幣31,000 元。
受保旅程：信用卡購買票據顯示出發日期開始至返回日期止，最長不得超過 180 天
保障內容：
旅程取消：最高保險金額達新台幣31,000 元
旅程縮短：最高保險金額達新台幣31,000 元
旅行延誤超過4小時：最高保險金額達新台幣3,100 元
錯過轉接交通：最高保險金額達新台幣3,100 元
更多活動資訊，可詳見活動網站
Visa活動頁：
					https://rebrand.ly/FY22VisaVIOffer

申請理賠頁：
					https://www-280.aig.com/visa/portal/il-apac/apac/zh-hk?step=RG001(須輸入旅人卡前8碼卡號查看保險細節及申請理賠)
備註：*旅行不便保險權益由Visa贊助並由AIG提供，相關活動詳情及限制請依Visa國際組織公告為準，活動相關更新本行恕不另行通知。


【 保費分期享 3.88% 起優惠利率 】

活動期限：即日起 至 2023/12/31
活動內容：滙豐旅人卡刷保費享不限金額哩程回饋
最長可分 30 期：不限公司險種，保費分期享 3.88% 起優惠利率，延後付款超輕鬆！
1,00


☆國泰 長榮航空聯名無限卡☆

一、年費

1. 正卡年費：NT$2,400
二、特色

1. 啟動倍數引擎 最優 NT$10 = 1 哩
2. 長榮航空官網、EVA Mobile APP 購票最優 95 折
3. 長榮航空機場貴賓室優惠價 NT$600 / 次

立即申請: 


☆優惠與權益☆

一、基本優惠

1. 哩程累積
   啟動倍數引擎 最優 NT$10 元 = 1 哩

【啟動倍數引擎 最優 NT$10 = 1 哩】

2. 旅遊訂房
   長榮航空官網、EVA Mobile APP 購票最優 95 折

【長榮航空官網、EVA Mobile APP 購票最優 95 折】

3. 旅遊訂房
   機場貴賓室與機場餐廳優惠價 NT$600 / 次

【長榮航空機場貴賓室優惠價 NT$600 / 次】

二、新卡友優惠

1. 首刷禮
   首刷贈 3,000 哩

【首刷贈 3,000 哩】

三、卡片須知

1. 年費
   正卡年費：NT$2,400

正卡年費：NT$2,400，附卡免年費
詳情請參閱國泰世華銀行官網信用卡年費規範

2. 循環利率
   6.75％~15％

國泰信用卡循環利率: 6.75％~15％（依國泰世華銀行信用評分制度定期評估，循環利率之基準日為104年9月1日）。
預借現金手續費：預借現金金額乘以3％加上新臺幣150元或美元5元，其他相關費用係依國泰世華銀行網站公告。
謹慎理財，信用至上

3. 申辦條件
   申請正卡須成年，年收入達 NT$60 萬

成年人可申請正卡
附卡申請人需為正卡申請人之父母、兄弟姊妹、配偶、配偶之父母或年滿 15 歲之子女
附卡申請人如未成年，須法定代理人（即父母/監護人）共同簽名同意
現職需滿一年且年收入達 NT$60 萬

☆國泰 長榮航空聯名御璽卡☆

一、年費

1. 首年免年費，附卡免年費
二、特色

1. 哩程加速器最優 NT$30 = 1 里數
2. 長榮航空官網、EVA Mobile APP 購票最優 95 折
3. 機場貴賓室與機場餐廳優惠價 NT$800 / 次

立即申請: 


☆優惠與權益☆

一、基本優惠

1. 哩程累積
   哩程加速器最優 NT$30 = 1 里數

【哩程加速器最優 NT$30 = 1 里數】

2. 旅遊訂房
   長榮航空官網、EV